<a href="https://colab.research.google.com/github/rgpereira-br/imersao-powerbi/blob/main/champions_imersao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import json
import pandas as pd
from datetime import datetime, date, timedelta
from google.cloud import bigquery
from google.api_core.exceptions import NotFound

In [3]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
API_KEY = '738632b8d635c579bff25a3653777ce8'
API_URL = 'https://v3.football.api-sports.io'
PROJECT_ID = 'betfullx-soccer'
DATASET_NAME = 'soccer'
FULL_LOAD_DATE = '2023-01-01'
LEAGUE = 2
SEASON = 2023
HEADERS = {
    'x-rapidapi-key': API_KEY
}

In [ ]:
endpoints = [
    {
        "table": "past_fixtures",
        "write_disposition": "WRITE_APPEND",
        "path": "fixtures",
        "quality_control": False,
        "params": {
            "league": LEAGUE,
            "season": SEASON
        },
        "incremental_load_params": {
            "from": "YYYY-MM-DD",
            "to": "YYYY-MM-DD"
        },
        "fields": [],
        "nested_fields": [
            "fixture.id",
            "fixture.referee",
            "fixture.timezone",
            "fixture.date",
            "fixture.timestamp",
            "fixture.periods.first",
            "fixture.periods.second",
            "fixture.venue.id",
            "fixture.venue.name",
            "fixture.venue.city",
            "fixture.status.long",
            "fixture.status.short",
            "fixture.status.elapsed",
            "league.id",
            "league.name",
            "league.country",
            "league.logo",
            "league.flag",
            "league.season",
            "league.round",
            "teams.home.id",
            "teams.home.name",
            "teams.home.logo",
            "teams.home.winner",
            "teams.away.id",
            "teams.away.name",
            "teams.away.logo",
            "teams.away.winner",
            "goals.home",
            "goals.away",
            "score.halftime.home",
            "score.halftime.away",
            "score.fulltime.home",
            "score.fulltime.away",
            "score.extratime.home",
            "score.extratime.away",
            "score.penalty.home",
            "score.penalty.away"
        ],
        "repeatable_fields": []
    },
    {
        "table": "future_fixtures",
        "write_disposition": "WRITE_TRUNCATE",
        "path": "fixtures",
        "quality_control": False,
        "params": {
            "league": LEAGUE,
            "season": SEASON,
            "to": "2099-12-31"
        },
        "incremental_load_params": {
            "from": "YYYY-MM-DD",
        },
        "fields": [],
        "nested_fields": [
            "fixture.id",
            "fixture.timezone",
            "fixture.date",
            "fixture.timestamp",
            "fixture.venue.id",
            "fixture.venue.name",
            "fixture.venue.city",
            "fixture.status.long",
            "fixture.status.short",
            "league.id",
            "league.name",
            "league.country",
            "league.logo",
            "league.flag",
            "league.season",
            "league.round",
            "teams.home.id",
            "teams.home.name",
            "teams.home.logo",
            "teams.away.id",
            "teams.away.name",
            "teams.away.logo"
        ],
        "repeatable_fields": []
    },
    {
        "table": "players",
        "write_disposition": "WRITE_TRUNCATE",
        "path": "players",
        "quality_control": True,
        "params": {
            "league": LEAGUE,
            "season": SEASON,
            "page": 1
        },
        "fields": [],
        "nested_fields": [
            "player.id",
            "player.name",
            "player.firstname",
            "player.lastname",
            "player.age",
            "player.birth.date",
            "player.birth.place",
            "player.nationality",
            "player.height",
            "player.weight",
            "player.injured",
            "player.photo"
        ],
        "repeatable_fields": []
    },
]

iterable_endpoints = {
    "past_fixtures": [
        {
            "table": "fixturesStatistics",
            "write_disposition": "WRITE_APPEND",
            "path": "fixtures/statistics",
            "query_param": {
                "fixture": "fixture.id"
            },
            "fixed_params": {},
            "fields": ["fixture"],
            "nested_fields": [
                "team.id",
                "team.name",
                "team.logo"
            ],
            "repeatable_fields": [
                "statistics"
            ]
        },
        {
            "table": "fixturesLineups",
            "write_disposition": "WRITE_APPEND",
            "path": "fixtures/lineups",
            "query_param": {
                "fixture": "fixture.id"
            },
            "fixed_params": {},
            "fields": [
                "fixture",
                "formation"
            ],
            "nested_fields": [
                "team.id"
            ],
            "repeatable_fields": [
                "startXI"
            ]
        }
    ]
}